In [13]:
import json
import yaml
import io
import os


f = open('./results/waypoint_maker.waypoint_clicker.json')
inferred = json.load(f)
handwritten_models_dict = dict()
handwritten_models_path = "../rostimer-evaluation/handwritten-models/"
handwritten_models_files = os.listdir(handwritten_models_path)
periodic_found = 0
periodic_not_found = 0
reactive_found = 0
reactive_not_found = 0
state_var_found = 0
state_var_unknown = 0
state_var_not_found = 0
state_transition_found = 0
state_transition_not_found = 0


for model_file_name in handwritten_models_files:
    with open(handwritten_models_path + model_file_name, 'r') as model_file:
        model_yml = yaml.safe_load(model_file)
        handwritten_models_dict[model_yml["package"] + "-" + model_yml["node"]] = model_yml

        inferred_path = f'./results/{model_yml["package"]}.{model_yml["node"]}.json'
        if not os.path.exists(inferred_path):
            print(f"ERROR! Didn't find {inferred_path}")
            continue
        inferred = json.load(open(inferred_path))
    
        print(inferred_path)    
        for pb in model_yml["periodic_behavior"]:
            found = False
            for pb_inferred in inferred["periodic_behavior"]:
                if pb_inferred["publisher"]["variable"] == pb["publisher"]["variable"]:
                    found = True
            if found:
                periodic_found += 1
                pb_inferred_freq = pb_inferred["rate"]
                pb_freq = pb["rate"]
                print(f"inferred frequency: {pb_inferred_freq}, handwritten frequency: {pb_freq}")
            else:
                print(f"Did not find periodic "+pb["publisher"]["variable"] + " in " + inferred_path)
                periodic_not_found += 1
        for rb in model_yml["reactive_behavior"]:
            found = False
            if "event" in rb:
                continue
            #print(rb)
            for rb_inferred in inferred["reactive_behavior"]:
                if rb["publisher"]["variable"] == rb_inferred["publisher"]["variable"] and rb["subscriber"]["callback"] == rb_inferred["subscriber"]["callback"]:
                    found = True
            if found:
                reactive_found += 1
            else:
                print(f"Did not find reactive "+str(rb) +  " in " + str(rb_inferred) + " file: " + inferred_path)
                reactive_not_found += 1
        for sv in model_yml["component_state_machine"]["state_variables"]:
            found = False
            for sv_inferred in inferred["potential_state_vars"]:
                if sv["name"] == sv_inferred["variable"]:
                    found = True
                    break
            if found:
                state_var_found += 1
                if sv_inferred["initialValue"]["kind"] == "unknown":
                    state_var_unknown += 1
                else: 
                    print("initialValue found!")
                    print(sv_inferred)
            else:
                print(f"Did not find state var "+str(sv) +  " in " + str(sv_inferred) + " file: " + inferred_path)
                state_var_not_found += 1

        if "transitions" in model_yml["component_state_machine"]:
            for st in model_yml["component_state_machine"]["transitions"]:
                found = False
                if "transitions" in inferred:
                    for st_inferred in inferred["transitions"]:
                        print(st_inferred)
                        if st["type"] == st_inferred["type"]:
                            if st["type"] == "message" and st["callback"] == st_inferred["callback"]:
                                found = True
                            if st["type"] == "interval" and st["interval"] == st_inferred["interval"]:
                                found = True
                            
                if found:
                    state_transition_found += 1

                else:
                    print(f"Did not find state transition "+str(st) +  " in " + str(st_inferred) + " file: " + inferred_path)
                    state_transition_not_found += 1                

print(f"periodic_not_found: {periodic_not_found}")
print(f"periodic_found: {periodic_found}")


print(f"reactive_not_found: {reactive_not_found}")
print(f"reactive_found: {reactive_found}")

print(f"state_var_not_found: {state_var_not_found}")
print(f"state_var_unknown: {state_var_unknown}")
print(f"state_var_found: {state_var_found}")

print(f"state_transition_found: {state_transition_found}")
print(f"state_transition_not_found: {state_transition_not_found}")

./results/viewers.scan_image_d_viewer.json
./results/waypoint_maker.waypoint_velocity_visualizer.json
Did not find reactive {'subscriber': {'callback': 'WaypointVelocityVisualizer::laneWaypointsArrayCallback', 'callback-location': 'waypoint_velocity_visualizer.cpp:198', 'subscribe-location': 'waypoint_velocity_visualizer.cpp:149', 'variable': 'WaypointVelocityVisualizer::lane_waypoints_array_sub_', 'topic': 'lane_waypoints_array', 'format': 'autoware_msgs/LaneArray', 'queue-size': 1}, 'publisher': {'variable': 'WaypointVelocityVisualizer::velocity_marker_pub_', 'call-location': 'waypoint_velocity_visualizer.cpp:266', 'advertise-location': 'waypoint_velocity_visualizer.cpp:161', 'topic': 'waypoints_velocity', 'format': 'visualization_msgs/MarkerArray', 'queue-size': 10}} in {'publisher': {'variable': 'velocity_marker_pub_'}, 'subscriber': {'callback': 'WaypointVelocityVisualizer::laneWaypointsArrayCallback'}} file: ./results/waypoint_maker.waypoint_velocity_visualizer.json
Did not find 